In [20]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn.metrics import log_loss, roc_auc_score, f1_score, accuracy_score

from mlfinlab.clustering.feature_clusters import get_feature_clusters

from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

In [12]:
DATA_PATH = 'input/'

In [13]:
BATCH_SIZE = 8192
EPOCHS = 200
LR = 1e-3
WEIGHT_DECAY = 1e-5
EARLYSTOP_NUM = 3
nFOLDS = 5
device = torch.device('cuda')

In [14]:
TRAIN = True
CACHE_PATH ='models/'
print('Reading input data...')
train = pd.read_csv(f'{DATA_PATH}train.csv')
train = train.query('date > 85').reset_index(drop=True)
print('Optimizing memory usage...')
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns})
print('Complete')
features = [c for c in train.columns if 'feature' in c]
f_mean = train[features[1:]].mean()
train[features] = train[features].fillna(method='ffill').fillna(0)
train = train.query('weight > 0').reset_index(drop=True)
train['action'] = (train['resp'] > 0).astype('int') #BCE Target
train['action_1'] = (train['resp_1'] > 0).astype('int')
train['action_2'] = (train['resp_2'] > 0).astype('int')
train['action_3'] = (train['resp_3'] > 0).astype('int')
train['action_4'] = (train['resp_4'] > 0).astype('int')

Reading input data...
Optimizing memory usage...
Complete


In [17]:
target_cols = [target for target in train.columns if 'action' in target]
if TRAIN:
    feat_cols= [col for col in features]
    
    train['cross_41_42_43'] = train['feature_41'] + train['feature_42'] + train['feature_43']
    
    train['cross_1_2'] = train['feature_1'] / (train['feature_2'] + 1e-5)
   
    feat_cols.extend(['cross_41_42_43','cross_1_2'])

In [16]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=1111)

In [44]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(feat_cols))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(feat_cols), hidden_size)
        nn.init.kaiming_normal_(self.dense1.weight,mode='fan_out')
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(feat_cols), hidden_size)
        nn.init.kaiming_normal_(self.dense2.weight,mode='fan_out')
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        nn.init.kaiming_normal_(self.dense3.weight,mode='fan_out')
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        nn.init.kaiming_normal_(self.dense4.weight,mode='fan_out')
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))
        nn.init.kaiming_normal_(self.dense5.weight,mode='fan_out')

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

In [3]:
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            self.save_checkpoint(epoch_score, model, model_path)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [4]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [5]:
def train_fn(model, optimizer, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        X,y = data['features'].to(device),data['label'].to(device)
        outputs = model(X)
        loss = loss_fn(outputs,y)
        loss.backward()
        optimizer.step()
        
        final_loss += loss.item()
    final_loss/=len(dataloader)
        
    return final_loss
        

In [6]:
def inference(model, dataloader, device,loss_fn=None):
    model.eval()
    preds = []
    val_loss = 0
    for data in dataloader:
        X,y = data['features'].to(device),data['label'].to(device)
        with torch.no_grad():
            outputs = model(X)
        if loss_fn:
            loss = loss_fn(outputs,y)
        
            val_loss += loss.item()
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
        
        
    preds = np.concatenate(preds).reshape(-1,len(target_cols))
    
    if loss_fn:
        val_loss/=len(dataloader)
    else:
        val_loss = None
    
    
    
    return val_loss, preds

In [7]:
class MarketDataset:
    def __init__(self, df):
        self.features = df[feat_cols].values

        self.label = df[target_cols].values.reshape(-1, len(target_cols))

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.features[idx], dtype=torch.float),
            'label': torch.tensor(self.label[idx], dtype=torch.float)
        }

In [8]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    # print('weight: ', weight)
    # print('resp: ', resp)
    # print('action: ', action)
    # print('weight * resp * action: ', weight * resp * action)
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [39]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [40]:
gkf = PurgedGroupTimeSeriesSplit(n_splits=5, group_gap=20)
splits = list(gkf.split(train[['resp']],groups=train['date'].values))

In [45]:
device = torch.device("cuda")
model = Model().to(device)

optimizer = torch.optim.Adam(model.parameters(),lr=LR,weight_decay=WEIGHT_DECAY)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='max',patience=5,verbose=True)

loss_fn = SmoothBCEwLogits(smoothing=0.005)

for _fold, (tr,te) in enumerate(splits):
    print(f'Fold: {_fold}')
    seed_everything(seed=1111+_fold)
    #torch.cuda.empty_cache()
    


    
    model_weights = f'{CACHE_PATH}/online_model_v8_{_fold}.pkl'
    es = EarlyStopping(patience=10,mode='max')
    
    train_set = MarketDataset(train_features.loc[tr])
    val_set = MarketDataset(train_features.loc[te])
    
    train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=False)
    valid_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=False)
    
    for epoch in (t:=trange(EPOCHS)):
        train_loss = train_fn(model,optimizer,loss_fn,train_loader,device)
        
        valid_loss, valid_preds = inference(model,valid_loader,device,loss_fn)
        
        roc_score = roc_auc_score(train.loc[te][target_cols].values,valid_preds)
        valid_preds = np.median(valid_preds,axis=1)
        valid_preds = np.where(valid_preds >= 0.5, 1, 0).astype(int)
        f1 = f1_score(train.loc[te][['action']].values,valid_preds)
        scheduler.step(roc_score)
        valid_u_score = utility_score_bincount(date=train.loc[te].date.values,weight=train.loc[te].weight.values,
                                               resp=train.loc[te].resp.values,action=valid_preds)
        nn.utils.clip_grad_norm_(model.parameters(),5)
        
        es(roc_score,model,model_path=model_weights)
        if es.early_stop:
            print('Early stopping')
            break
         
        t.set_description("Train loss {} Valid loss {} ROC_score {} f1_score {} Utility_score {}".format(train_loss,valid_loss,roc_score,f1,valid_u_score))
    torch.save(model.state_dict(), f'{CACHE_PATH}/model_v8_{_fold}.pkl')

Fold: 0


Train loss 3.1047840780682034 Valid loss 1.545263487717201 ROC_score 0.5095102489058656 f1_score 0.43946959902724575 Utility_score 30.64448693665193:   0%|          | 1/200 [00:10<35:10, 10.61s/it]

Validation score improved (-inf --> 0.5095102489058656). Saving model!


Train loss 1.8096604082319472 Valid loss 0.9916201706590324 ROC_score 0.5117753043300762 f1_score 0.5219295542599828 Utility_score 115.42403041933945:   1%|          | 2/200 [00:21<35:08, 10.65s/it]

Validation score improved (0.5095102489058656 --> 0.5117753043300762). Saving model!


...

EarlyStopping counter: 8 out of 10


Train loss 0.7627543405526214 Valid loss 0.7032493841953766 ROC_score 0.5397267063798387 f1_score 0.5864357675404496 Utility_score 1281.6230453614596:  14%|█▍        | 29/200 [19:20<1:52:40, 39.54s/it]

EarlyStopping counter: 9 out of 10


Train loss 0.7627543405526214 Valid loss 0.7032493841953766 ROC_score 0.5397267063798387 f1_score 0.5864357675404496 Utility_score 1281.6230453614596:  14%|█▍        | 29/200 [19:59<1:57:52, 41.36s/it]

EarlyStopping counter: 10 out of 10
Early stopping
